## **Bibliotecas**

In [1]:
# API
import requests
import json
import csv

# Manipulação de dados
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

# Normalização
from sklearn.preprocessing import MinMaxScaler

# MLP
from sklearn.neural_network import MLPClassifier

# Cross Validation
from sklearn.model_selection import cross_val_predict

# Amostra dos dados
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## **API Spotify**

In [3]:
class SpotifyApi:
    def __init__(self):
        # informações da aplicação do spotify
        self._clientId = '9956a83a95c94d72a6c04c7cfa63ec52'
        self._clientSecret = '1a29d0e37da34ffc92a68a5527881a53'

        # token (que expira após um tempo)
        self.accessToken = self.getAccessToken()

    def request(self, url, rType, headers = {}, params = {}):
        '''
        Realiza uma requisição e verifica se deu certo
        '''

        if rType == 'get':
            headers['Authorization'] = 'Bearer ' + str(self.accessToken)
            r = requests.get(url, headers=headers, params=params)
        elif rType == 'post':
            r = requests.post(url, headers=headers, params=params)

        # checa se a requisição deu certo
        rSC = r.status_code
        if rSC >= 200 and rSC < 300:
            return r
        else:
            print('Erro ' + str(r.status_code) + ': ' + r.reason)
            return None

    def uriToId(self, uri):
        '''
        Se necessário, isola e retorna o ID do URI
        '''

        if uri.startswith('spotify'):
            uri = uri.split(':')[-1]
        return uri

    def getAccessToken(self):
        '''
        gera um novo token e o armazena
        '''

        url = 'https://accounts.spotify.com/api/token'

        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        params = {'grant_type': 'client_credentials', 'client_id': self._clientId, 'client_secret': self._clientSecret}

        r = self.request(url, 'post', headers, params)
        rJson = r.json()

        return rJson['access_token']

    def getPlaylist(self, playlistId, fields = 'id, name, owner(display_name), tracks.items(track(artists(id, name), name, id))'):
        '''
        Retorna dados de uma playlist específica
        '''

        if not fields == '': fields += ', type'

        # codifica fields para funcionar no link
        fields = fields.replace(',', '%2C')
        fields = fields.replace(' ', '+')
        fields = fields.replace('(', '%28')
        fields = fields.replace(')', '%29')

        # pega o ID da playlist
        playlistId = self.uriToId(playlistId)

        # pega os dados da playlist
        url = 'https://api.spotify.com/v1/playlists/' + playlistId + '?fields=' + fields
        r = self.request(url, 'get')

        # checa se deu erro
        if not r:
            print('Playlist dada é privada')
        else:
            return r.json()

    def getTrack(self, trackID):
        '''
        Retorna dados de uma track (música) específica
        '''

        trackId = self.uriToId(trackID)
        url = 'https://api.spotify.com/v1/tracks/' + trackId
        r = self.request(url, 'get')

        if not r:
            print('Música com ID '+ str(trackId) + 'não encontrada')
        else:
            return r.json()

    def getArtist(self, artistId):
        '''
        Retorna dados de um artista específico
        '''

        artistId = self.uriToId(artistId)
        url = 'https://api.spotify.com/v1/artists/' + artistId
        r = self.request(url, 'get')

        if not r:
            print('Artista com ID '+ str(trackId) + 'não encontrada')
        else:
            return r.json()

    def printJson(self, j):
        '''
        Printa o Json
        '''

        jsonStr = json.dumps(j, indent=1)
        print(jsonStr)

    def playlistToCsv(self, p, playlistNum):
        '''
        Printa os dados da playlist e passa para arquivo CSV
        '''

        try:
            if p['type'] != 'playlist':
                print('Objeto não é uma playlist')
                return
        except:
            print('Não foi possível verificar a playlist pelo atributo "type"')
            return

        print('Playlist ' + p['name'])
        print('por ' + p['owner']['display_name'])
        print('-----')

        playlistName = 'playlist' + str(playlistNum) + '.csv'
        with open(playlistName, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow(['id', 'name', 'popularity', 'release date', 'duration', 'explicit', 'artists', 'genres', 'class'])
            for items in p['tracks']['items']:
                track = items['track']
                data = self.getTrackData(track, playlistNum)
                writer.writerow(data)

    def getTrackData(self, track, num_playlist = 0):
        '''
        Recebe um json de uma track e retorna os seus dados em uma lista
        '''

        trackId = track['id']

        trackName = track['name']
        print(trackName, end=' - ')

        trackDate = track['album']['release_date']
        print(trackDate, end=' - ')

        trackDur = track['duration_ms']
        print(trackDur, end=' - ')

        trackPopularity = track['popularity']
        print(trackPopularity, end=' - ')

        trackExplicit = track['explicit']
        print(trackExplicit, end=' - ')

        artists = track['artists']
        list_artists = []
        list_genres = []
        for i, artist in enumerate(artists):
            artistId = artist['id']
            artistJson = self.getArtist(artistId)
            artistName = artistJson['name']
            artistGenres = artistJson['genres']

            # coloca virgula entre os artistas
            if i == len(artists) - 1: end = '\n'
            else: end = ', '
            print(artistName, end = ' - ')
            print(artistGenres, end = end)

            list_artists.append(artistName)
            list_genres.append(artistGenres)

        mainArtist = list_artists[0]
        mainGenre = list_genres[0]

        data = [trackId, trackName, trackPopularity, trackDate, trackDur, trackExplicit, mainArtist, mainGenre]
        if num_playlist == 1:
            data.append('playlist 1')
        elif num_playlist == 2:
            data.append('playlist 2')
        else:
            data.append('sem playlist')
        return data

## **Tratamento de dados**


### Gerenciamento dos arquivos CSV

---



In [4]:
def writePlaylist(playlistId, playlistNum):
    '''
    Escreve os dados de uma playlist em um arquivo CSV
      playlistId: Id da playlist no spotify
      playlistNum: número da playlist no treinamento, 1 ou 2
    '''

    api = SpotifyApi()

    print("\nPlaylist " + str(playlistNum))

    playlist = api.getPlaylist(playlistId, '')
    api.playlistToCsv(playlist, playlistNum)

In [5]:
def writeDataBase(playlistId1, playlistId2):
    '''
    Junta as duas playlists em um só aquivo CSV (database.csv)
    '''

    writePlaylist(playlistId1, 1)
    writePlaylist(playlistId2, 2)

    all_filenames = ['playlist1.csv', 'playlist2.csv']

    # combina todos os arquivos da lista
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])

    # exporta para CSV
    combined_csv.to_csv("database.csv", index=False, encoding='utf-8-sig')

### Processamento dos dados para a Rede Neural

---



In [6]:
def prepDataFrame(df):
  '''
  Prepara os dados para o uso na rede neural:
    Retira o id e nome das tracks
    Transforma os dados string para int
  '''

  df = df.drop('id', axis=1)
  df = df.drop('name', axis=1)

  # traduz os dados string para numérico
  for var in ['release date', 'explicit', 'artists', 'genres', 'class']:
      temp = np.array(df[var]).reshape(-1, 1)
      df[var] = enc.fit_transform(temp)

  return df

Normalização

In [7]:
def normalizeDataFrame(df):
  '''
  Normaliza os dados
  '''

  features = df.drop(['class'], axis=1)

  scaler = MinMaxScaler()
  norm_data = scaler.fit_transform(features)
  df_norm_data = pd.DataFrame(norm_data)

  df_norm_data['class'] = df['class']

  return df_norm_data

Processamento total dos dados

In [8]:
def dataProcessing(df):
  '''
  Prepara o dataframe para o treinamento e processamento
  '''

  df = prepDataFrame(df)
  #print(df)
  return normalizeDataFrame(df)

## **MLP**

In [9]:
class MLP:
    def __init__(self, df, printing = False):
        self.mlp = MLPClassifier(max_iter=1000, solver='adam', hidden_layer_sizes= 106, activation='relu', random_state=1)
        self.df = df
        self.dfNorm = dataProcessing(df)
        self.entries = self.dfNorm.drop(['class'], axis=1)
        self.classes = self.dfNorm['class']
        self.spotifyApi = SpotifyApi()

    def classifier(self, cv_folds=5):
        '''
        Treina e testa a MLP com validação cruzada
        '''

        x = self.entries
        y = self.classes

        # previsões
        predictions = cross_val_predict(self.mlp, x, y, cv=cv_folds)

        # matriz de confusão e métricas
        self.confusionMatrix(predictions)

        # previsões erradas
        self.getWrongResults(predictions)

    def confusionMatrix(self, predictions):
        '''
        Printa a matriz de confusão e métricas do classificador
        '''

        x = self.entries
        y = self.classes

        # matriz de confusão
        matriz_confusao = metrics.confusion_matrix(y, predictions)

        plt.title("Confusion Matrix")
        df_cm = pd.DataFrame(matriz_confusao, range(2), range(2))
        ax = sns.heatmap(df_cm, annot=True, fmt='g', cmap='crest')
        labels = ['playlist 1', 'playlist 2']
        ax.set_xticklabels(labels)
        ax.set_yticklabels(labels)
        plt.xlabel('Predicted')
        plt.ylabel('True')

        # métricas
        print('Métricas:')
        print(metrics.classification_report(y, predictions, target_names=labels))

    def getWrongResults(self, predictions):
        '''
        Printa quais músicas o classificador errou
        '''

        x = self.entries
        y = self.classes

        # index da lista predictions
        predIndex = 0

        #contador
        count = 1

        print('\nErros:')
        # roda por cada item nos dados completos
        for i in y.axes[0]:
            # index do item no database.csv
            dfIndex = i

            # playlist dada pela MLP para o item
            if predictions[predIndex] == 0:
                playlistPred = 'playlist 1'
            else:
                playlistPred = 'playlist 2'

            # iloc devolve a linha com index dfIndex do DataFrame (database.csv)
            track = self.df.iloc[dfIndex]

            # printa os exemplos que classificador errou
            if playlistPred != track['class']:
                print(f'Track {count}')
                count += 1
                print(track)
                playlistTrue = track['class']
                print(f'True: {playlistTrue} \nPredicted: {playlistPred}')
                print()

            # faz o index do predictions acompanhar o index do yTest
            predIndex += 1

## **Main**

### Criação dos arquivos CSV
---

> rodar apenas quando alguma playlist sofrer alteração

In [11]:
playlist1 = 'spotify:playlist:6O6nD8HLgRsvNRFU1pIYYt'
playlist2 = 'spotify:5joN8srNzN7Pe8qHyThnnv'

writeDataBase(playlist1, playlist2)


Playlist 1
Playlist so as boa.
por Lucas
-----
Sky Fall - 2014-12-16 - 318093 - 58 - True - Travis Scott - ['hip hop', 'rap', 'slap house'], Young Thug - ['atl hip hop', 'atl trap', 'gangster rap', 'melodic rap', 'rap', 'trap']
Animais - 2023-05-08 - 183652 - 56 - True - Yunk Vino - ['trap brasileiro'], Thermo Beats - [], Labbel Rec - ['trap brasileiro']
Só Quando Ela Me Quer - 2020-02-14 - 228012 - 59 - False - Yunk Vino - ['trap brasileiro'], Jaya Luuck - [], Duzz - [], Rubro - []
Red Dot - 2023-03-21 - 128000 - 59 - True - Yunk Vino - ['trap brasileiro'], Caio Passos - ['trap funk'], Labbel Rec - ['trap brasileiro']
Solta Minha Blusa - 2023-07-28 - 117500 - 67 - True - pumapjl - ['drill brasileiro'], SonoTWS - ['lo-fi brasileiro']
Fashion - 2020-06-26 - 150557 - 40 - True - Yunk Vino - ['trap brasileiro'], Nagalli - ['trap brasileiro']
Nightcrawler (feat. Swae Lee & Chief Keef) - 2015-09-04 - 321560 - 77 - True - Travis Scott - ['hip hop', 'rap', 'slap house'], Swae Lee - ['melodic

### MLP
---

In [ ]:
df = pd.read_csv('database.csv')
mlp = MLP(df)
mlp.classifier()

: 